# Segmenting and Clustering Neighborhoods in Toronto

By Tianji Cai

## Introduction

In this notebook, we will use **k-means clustering** algorithm to segment and cluster neighborhoods in Toronto. The neighborhood data will be obtained from this [Wikipedia page](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M). After cleaning and transforming the data, we will use the Foursquare API to get the most common venue categories in each neighborhood and then use this feature to group the neighborhoods into clusters, where k-means clustering algorithm will be used. Finally, the Folium library is used to visualize the neighborhoods in Toronto and their emerging clusters.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Scrape the Wikipedia Page and Transform the Data into a __pandas__ dataframe</a>

2. <a href="#item2">Get the Location Data for Each Neighborhood</a>

3. <a href="#item3">Explore and Cluster the Neighborhoods in Toronto</a>

4. <a href="#item3">Conclusions</a>

</font>
</div>

First, we download the necessary libraries before starting any work.

In [8]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import geocoder # import geocoder

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup # library for web scraping

!conda install -c conda-forge lxml --yes

import requests # library for requesting webpage

import csv # library for working with csv file

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


## 1. Scrape the Wikipedia Page and Transform the Data into a __pandas__ dataframe

We first scrape the webpage and download the table from Wikipedia into a pandas dataframe.

In [132]:
# Get the source file from Wikipedia
WIKI_URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(WIKI_URL).text 

# Convert the source file into a beautifulsoup object for scraping
soup = BeautifulSoup(source, 'lxml')
# print(soup)

# Get the relevant table
table_classes = {"class": ["sortable", "plainrowheaders"]}
wikitable = soup.find("table", table_classes)
#print(wikitable)

# Convert the table into a pandas dataframe
neighborhoods = pd.read_html(str(wikitable))
neighborhoods = pd.DataFrame(neighborhoods[0])
neighborhoods

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


Next, we clean the data.

In [133]:
# Rename the columns 
neighborhoods = neighborhoods.rename(columns={"Postcode": "PostalCode","Neighbourhood": "Neighborhood"})

# Drop the cells with a borough that is 'Not assigned'.
neighborhoods = neighborhoods[neighborhoods['Borough'] != 'Not assigned'].reset_index(drop=True)

# Combine the neighborhoods with the same postal code into one row, separated by comma 
neighborhoods = neighborhoods.groupby('PostalCode', as_index=False).agg({'Borough': 'min','Neighborhood': ', '.join})

# For cells that have a borough but a Not assigned neighborhood, its neighborhood will be the same as the borough.
neighborhoods.loc[neighborhoods['Neighborhood'] =='Not assigned', 'Neighborhood'] = neighborhoods.loc[neighborhoods['Neighborhood'] =='Not assigned', 'Borough']

# Inspect the result
neighborhoods

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [134]:
# Print the number of rows of the dataframe
print("The shape of the dataframe neighborhoods is: ", neighborhoods.shape)

The shape of the dataframe neighborhoods is:  (103, 3)


## 2. Get the Location Data for Each Neighborhood

We now get the latitude and the longitude coordinates of each neighborhood based on its postal code using geocoder. 

Note: Since geocoder is not very stable, we need to run a while loop for each postal code in order to make sure that we get the coordinates for all of our neighborhoods.

In [5]:
# Ignore this cell

"""
nrows = neighborhoods.shape[0] # Get the total number of rows in neighborhoods
ncol = neighborhoods.shape[1] # Get the total number of columns in neighborhoods
postal_code_all = neighborhoods['PostalCode'] # Get all postal codes in neighborhoods

postal_code = postal_code_all[0]
print(postal_code)
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
    print('lat, lng None')
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng
    print('lat_lng_coords')

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
print(latitude, longitude)
 
"""

"""
# Create two blank series to store the latitude and longitude values for all postal codes
latitude_all = pd.Series([])
longitude_all = pd.Series([]) 

for i in range(nrows)
    postal_code = postal_code_all[i]
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    
    latitude_all[i] = latitude
    longitude_all[i] = longitude
    
    
neighborhoods.insert(ncol, 'Latitude', latitude_all, allow_duplicates = False)
neighborhoods.insert(ncol+1, 'Longitude', longitude_all, allow_duplicates = False)
"""        

"\n# Create two blank series to store the latitude and longitude values for all postal codes\nlatitude_all = pd.Series([])\nlongitude_all = pd.Series([]) \n\nfor i in range(nrows)\n    postal_code = postal_code_all[i]\n    # initialize your variable to None\n    lat_lng_coords = None\n\n    # loop until you get the coordinates\n    while(lat_lng_coords is None):\n      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))\n      lat_lng_coords = g.latlng\n\n    latitude = lat_lng_coords[0]\n    longitude = lat_lng_coords[1]\n    \n    latitude_all[i] = latitude\n    longitude_all[i] = longitude\n    \n    \nneighborhoods.insert(ncol, 'Latitude', latitude_all, allow_duplicates = False)\nneighborhoods.insert(ncol+1, 'Longitude', longitude_all, allow_duplicates = False)\n"

Unfortunately, the above method using geocoder fails because it takes way too long to fetch values from geocoder. Therefore, we switch to our plan B and download the csv file with latitude, longitude data from the [source](https://cocl.us/Geospatial_data) given in the course.

In [135]:
# Get the csv, transform it into a pandas df and clean it
lat_lng = pd.read_csv('https://cocl.us/Geospatial_data')
lat_lng = lat_lng.rename(columns={"Postal Code": "PostalCode"})
lat_lng.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [136]:
# Merge the two dataframes (neighborhoods, lat_lng) into one complete df that will be used in subsequent analysis
neighborhoods = pd.merge(neighborhoods, lat_lng, on='PostalCode')
print('The shape of neighborhoods is: ', neighborhoods.shape)
neighborhoods

The shape of neighborhoods is:  (103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


#### Now, we've had our complete data stored in neighborhoods! Let's start exploring it!

## 3. Explore and Cluster the Neighborhoods in Toronto

### 3.1 Map of Neighborhoods in Toronto

First, let's create a map of Toronto with all the neighborhoods superimposed on top, using folium. This will help us to get some intuition about the geography of Toronto.

In [16]:
# Get the latitude and longitude of Toronto, using geopy library.
address = 'Toronto, ON, CA'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [19]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}: {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### 3.2 Get Venues in Toronto from Foursquare API

Next, let's get top 100 venues within a radius of 500 meters of each neighborhood from Foursquare API.

In [116]:
# Define Foursquare credentials and version
CLIENT_ID = '5YANWUWTILTM15H3CSPEGY4KEKNKUHEO53KMQKV003KIKGTB' 
CLIENT_SECRET = 'UMSQMKIRVPLXMIDJG2HBHTVOSDSNPS0ZJFB0ZMLOOBXRNKZM' 
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5YANWUWTILTM15H3CSPEGY4KEKNKUHEO53KMQKV003KIKGTB
CLIENT_SECRET:UMSQMKIRVPLXMIDJG2HBHTVOSDSNPS0ZJFB0ZMLOOBXRNKZM


In [117]:
# Define a function to get relevant information about venues in a neighborhhod.
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [118]:
# Get the venues for all the neighborhoods in Toronto
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The D

Let's explore the venues in each neighborhood returned by Foursquare.

In [119]:
# Inspect our dataframe toronto_venues
print('The shape of toronto_venues is: ', toronto_venues.shape)
toronto_venues.head()

The shape of toronto_venues is:  (2249, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [120]:
# Check how many venues were returned for each neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",11,11,11,11,11,11
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Downsview North, Wilson Heights",17,17,17,17,17,17
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
Berczy Park,56,56,56,56,56,56


In [121]:
# Find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 275 uniques categories.


#### We have now obtained all the relevant venues information for each neighborhood and stored them in the dataframe toronto_venues!

### 3.3 Explore and Analyze Each Neighborhood

Create a dataframe which gives the number of venues in each category for every neighborhood.

In [122]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
cols = toronto_onehot.columns.tolist()
cols.insert(0, cols.pop(cols.index('Neighborhood')))
toronto_onehot = toronto_onehot.reindex(columns=cols)

print('The shape of toronto_onehot is: ', toronto_onehot.shape)
toronto_onehot.head()

The shape of toronto_onehot is:  (2249, 275)


,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [123]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print('The shape of toronto_grouped is ', toronto_grouped.shape)
toronto_grouped.head()

The shape of toronto_grouped is  (100, 275)


,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04,0.0,0.0,0.

Ha, we've lost 3 neighborhoods! The reason is that three neighborhoods (Upper Rouge; Newtonbrook, Willowdale; Islington Avenue) do not have venues returned by Foursquare. Let's drop these three neighborhoods from *neighborhoods* all together.

In [139]:
neighborhoods_with_venues = neighborhoods[neighborhoods['Neighborhood'].isin(toronto_grouped['Neighborhood'])].reset_index(drop=True)
print('The new neighborhoods_with_venues has shape: ', neighborhoods_with_venues.shape)
neighborhoods_with_venues

The new neighborhoods_with_venues has shape:  (100, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


Let's find out the top 5 most common venue categories in each neighborhood.

In [47]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2      Thai Restaurant  0.04
3           Steakhouse  0.04
4  American Restaurant  0.04


----Agincourt----
                venue  freq
0      Sandwich Place  0.25
1              Lounge  0.25
2      Breakfast Spot  0.25
3        Skating Rink  0.25
4  Miscellaneous Shop  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                 venue  freq
0           Playground   0.5
1                 Park   0.5
2   Mexican Restaurant   0.0
3                Motel   0.0
4  Monument / Landmark   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store  0.18
1             Pharmacy  0.09
2          Pizza Place  0.09
3           Beer Store  0.09
4  Fried Chicken Joint  0.09


----Alderwood, Long Branch----
            venue  freq
0     Pizza P

               venue  freq
0     Discount Store  0.25
1  Convenience Store  0.12
2        Bus Station  0.12
3   Department Store  0.12
4        Coffee Shop  0.12


----East Toronto----
               venue  freq
0  Convenience Store   0.4
1        Coffee Shop   0.2
2      Metro Station   0.2
3               Park   0.2
4  Accessories Store   0.0


----Emery, Humberlea----
                 venue  freq
0       Baseball Field   1.0
1    Accessories Store   0.0
2        Movie Theater   0.0
3                Motel   0.0
4  Monument / Landmark   0.0


----Fairview, Henry Farm, Oriole----
                  venue  freq
0        Clothing Store  0.13
1  Fast Food Restaurant  0.07
2           Coffee Shop  0.06
3            Restaurant  0.04
4         Women's Store  0.03


----First Canadian Place, Underground city----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.08
2                Hotel  0.06
3           Restaurant  0.05
4  American Restaurant  0.04


----Flem

                venue  freq
0         Coffee Shop  0.10
1                Café  0.08
2         Pizza Place  0.05
3  Italian Restaurant  0.05
4    Sushi Restaurant  0.05


----Ryerson, Garden District----
                       venue  freq
0                Coffee Shop  0.09
1             Clothing Store  0.09
2                       Café  0.04
3             Cosmetics Shop  0.03
4  Middle Eastern Restaurant  0.03


----Scarborough Village----
                       venue  freq
0                 Playground   0.5
1          Convenience Store   0.5
2  Middle Eastern Restaurant   0.0
3              Movie Theater   0.0
4                      Motel   0.0


----Silver Hills, York Mills----
                        venue  freq
0                   Cafeteria   1.0
1  Modern European Restaurant   0.0
2                 Music Store   0.0
3                      Museum   0.0
4               Movie Theater   0.0


----St. James Town----
            venue  freq
0     Coffee Shop  0.07
1      Restaurant  0.06

Create a new dataframe that displays the top 10 venue categories for each neighborhood.

In [141]:
# First define a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [163]:
# Create the new dataframe
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

print('The shape of neighborhoods_venues_sorted is: ', neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

The shape of neighborhoods_venues_sorted is:  (100, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,American Restaurant,Restaurant,Hotel,Bakery,Bar,Gym
1,Agincourt,Breakfast Spot,Lounge,Sandwich Place,Skating Rink,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Fried Chicken Joint,Coffee Shop,Sandwich Place,Discount Store,Fast Food Restaurant,Beer Store,Japanese Restaurant,Pharmacy
4,"Alderwood, Long Branch",Pizza Place,Skating Rink,Pharmacy,Gym,Pub,Sandwich Place,Coffee Shop,German Restaurant,Dumpling Restaurant,Gift Shop


#### We've now created a dataframe *neighborhoods_venues_sorted* that stores the 10 most common venue categories for each neighborhood. We also have another dataframe *toronto_grouped* that includes the mean of the frequency of occurrence of each category for every neighborhood. This later dataframe *toronto_grouped* will be used to cluster neighborhoods.

### 3.4 Cluster Neighborhoods

Run *k*-means to cluster the neighborhoods into 5 clusters. Try different numbers of clusters to play around and choose a proper number of clusters so as to generate clusters with more equal size. It seems that 5 is a pretty good choice.

In [172]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 4, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 2, 0, 0,
       0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 4, 0, 0, 4,
       0, 4, 0, 0, 0, 0, 4, 0, 4, 4, 4, 0, 0, 4, 4, 0, 0, 0, 0, 0, 0, 0,
       0, 4, 0, 0, 0, 4, 0, 0, 0, 0, 0, 3], dtype=int32)

Let's create a new dataframe that includes the cluster label as well as the top 10 venues for each neighborhood.

In [173]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head()

toronto_merged = neighborhoods_with_venues

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

print('The shape of toronto_merged is: ', toronto_merged.shape)
toronto_merged.head() 

The shape of toronto_merged is:  (100, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,4,Fast Food Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,College Gym
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0,Bar,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Dim Sum Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Pizza Place,Mexican Restaurant,Electronics Store,Rental Car Location,Medical Center,Breakfast Spot,Diner,Discount Store,Dog Run,Doner Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Athletics & Sports,Caribbean Restaurant,Bakery,Thai Restaurant,Bank,Fried Chicken Joint,Hakka Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant


Let's now visualize the resulting clusters.

In [174]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 3.5 Examine the Clusters

Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster.

#### Cluster 1: City Life Cluster

In [180]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",0,Bar,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Dim Sum Restaurant
2,Scarborough,"Guildwood, Morningside, West Hill",0,Pizza Place,Mexican Restaurant,Electronics Store,Rental Car Location,Medical Center,Breakfast Spot,Diner,Discount Store,Dog Run,Doner Restaurant
3,Scarborough,Woburn,0,Coffee Shop,Korean Restaurant,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
4,Scarborough,Cedarbrae,0,Athletics & Sports,Caribbean Restaurant,Bakery,Thai Restaurant,Bank,Fried Chicken Joint,Hakka Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
6,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",0,Discount Store,Department Store,Hobby Shop,Train Station,Convenience Store,Bus Station,Coffee Shop,Empanada Restaurant,Ethiopian Restaurant,Event Space
7,Scarborough,"Clairlea, Golden Mile, Oakridge",0,Bakery,Bus Line,Fast Food Restaurant,Intersection,Food Truck,Bus Station,Soccer Field,Cosmetics Shop,Coworking Space,Falafel Restaurant
8,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",0,Motel,American Restaurant,Yoga Studio,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
9,Scarborough,"Birch Cliff, Cliffside West",0,General Entertainment,College Stadium,Farm,Café,Skating Rink,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop
10,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",0,Indian Restaurant,Vietnamese Restaurant,Furniture / Home Store,Latin American Restaurant,Chinese Restaurant,Pet Store,Empanada Restaurant,Electronics Store,Ethiopian Restaurant,Eastern European Restaurant
11,Scarborough,"Maryvale, Wexford",0,Auto Garage,Smoke Shop,Middle Eastern Restaurant,Breakfast Spot,Sandwich Place,Bakery,Donut Shop,Discount Store,Dog Run,Doner Restaurant


This is the most popular cluster which includes the majority of the neighborhoods (red dots in the map above). The most common venues in the neighborhoods in this cluster are __Restaurants__, __Shops and Stores__, and all sorts of other everyday places. The neighborhoods in this cluster seem like a great place to live and the venues there provide people with an easy and convenient city life. For this reason, I name it the __City Life Cluster__. Personally, I'd prefer moving into one of the neighborhoods in this cluster if I come to Toronto someday.  

#### Cluster 2: Bus Line Cluster

In [181]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
97,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",1,Bus Line,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Farmers Market


This is a lonely cluster, with only one neighborhood (the purple dot). The defining feature is the __Bus Line__, which is the 1st most common venue in this cluster yet seldom appears among the top 10 most common venues in any of the other clusters. 

#### Cluster 3: Baseball Field Cluster

In [182]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
89,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",2,Baseball Field,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Yoga Studio,Dim Sum Restaurant
94,North York,"Emery, Humberlea",2,Baseball Field,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Yoga Studio,Dim Sum Restaurant


There're only two neighborhoods in this cluster (blue dots). The characteristic feature are __Baseball Field__ and __Eastern European Restaurant__. Yet, since Baseball Field is the 1st most common venue in this cluster and seldom makes its appearance in other clusters, I prefer choosing __Baseball Field__ as the defining feature of this cluster. So if you're a baseball fan, maybe you should consider living in these neighborhoods. 

#### Cluster 4: Bank Cluster

In [178]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,North York,3,Park,Bank,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
91,Etobicoke,3,Bank,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Dim Sum Restaurant


The most common venues in the two neighborhoods in this cluster (green dots) is __Bank__. None of the neighborhoods in other clusters have that many banks. So maybe it's the financial center of Toronto?

#### Cluster 5: Natural Resort Cluster

In [179]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,4,Fast Food Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,College Gym
5,Scarborough,4,Convenience Store,Playground,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
14,Scarborough,4,Playground,Park,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
19,North York,4,Cafeteria,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop
23,North York,4,Food & Drink Shop,Fast Food Restaurant,Park,Bus Stop,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
28,North York,4,Airport,Park,Bus Stop,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
38,East York,4,Convenience Store,Metro Station,Coffee Shop,Park,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
42,Central Toronto,4,Bus Line,Park,Swim School,Lake,Dim Sum Restaurant,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant
46,Central Toronto,4,Playground,Restaurant,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
48,Downtown Toronto,4,Park,Playground,Trail,Yoga Studio,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run


This is the second most populous cluster (orange dots). The most common venues in this cluster seem to be __Park__, __Playground__, __Trail__, and other natural resorts. So I would say the defining feature is __Park__ and name this cluster __Natural Resort Cluster__. If you're a person who loves nature, maybe you should consider moving into the neighborhoods in this cluster! 

## Conclusions

In this notebook, we've used __k-means algorithm__ to group the neighborhoods in Toronto into 5 clusters. The neighborhood data is scraped from __Wikipedia__ and the venue data obtained from __Foursquare__. Because there're three neighborhoods (Upper Rouge in Scarborough; Newtonbrook, Willowdale in North York; Islington Avenue in Etobicoke) that do not have venue data from Foursquare, we have to exclude them in our analysis. The remaining neighborhoods are clustered into 5 groups based on their similarity of frequencies of different venue categories. 

Based on their defining features, the five clusters are given the names __City Life Cluster__, __Bus Line Cluster__, __Baseball Field Cluster__, __Bank Cluster__, and __Natural Resort Cluster__, respectively, with the __City Life Cluster__ and the __Natural Resort Cluster__ including the majority of neighborhoods in Toronto.  

The main __shortcoming__ of this analysis is that the sizes of the clusters vary too much, with two clusters containing nearly all the neighborhoods whereas the other three clusters including only one or two neighborhoods. This unequal size of clusters suggests that k-means clustering may not be the optimal machine learning algorithm for this problem. In the future, more algorithms can be used and then compared with each other to yield the best analysis. 

One __general commercial application__ of the analysis done here can be a __recommendation system__ for people who plan to move into a new city and are looking for a suitable neighborhood to live. Based on the preferences of the customers, the system can then recommend a neighborhood in the cluster that best matches their preferences. Indeed, we can build a website and offer this service for free (or even make money)!

#### Author: Tianji Cai      
#### Email: tianjiC@outlook.com